# OES_Transition_Identification

Written by: Ketong Shao and Angelo D. Bonzanini

Date: October 2021

* Reduce the OES spectra dimension using the trained Autoencoder
* Hyperparamter determination of Random forest using Bayesian optimization and 5-fold cross validation
* Examine the prediction accuracies using confusion matrices

## Imports

In [2]:
# General
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import tensorflow as tf
seed=0
np.random.seed(seed) # fix random seed
tf.random.set_seed(seed)

# Mount Google Drive
#from google.colab import drive
#drive.mount('/content/gdrive')

## Load the trained Autoencoder and reduce the OES data dimension

In [3]:
# The OES spectrum are first normalized by deviding their largest peaks.
# Therefore, each spectra has the highest intensity value as 1.
spec_regen = np.genfromtxt('corrp_specs0.001.csv',delimiter=',')
trans_id = np.genfromtxt('OES_true_labels.csv',delimiter=',')
# Load the traiend autoencider
autoencoder = tf.keras.models.load_model('autoencoder_model')

In [4]:
# Split the data into training and test dataset, which are the same as in the previous .ipynb script.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(spec_regen, trans_id, test_size=0.1, random_state=42)

# Build the scaler for OES data. Sometimes, 'StandardScaler' could be a good choice.
# It is not necessary and reasonable to normalize the output labels, which are just 1s or 0s.
from sklearn.preprocessing import MinMaxScaler
scaler_OES = MinMaxScaler()
scaled_Xtrain = scaler_OES.fit_transform(X_train)
scaled_Xtest = scaler_OES.transform(X_test)

# Compress the OES spectra data
X_train_auto = autoencoder.encoder(scaled_Xtrain)
Y_train_auto = y_train

X_test_auto = autoencoder.encoder(scaled_Xtest)
Y_test_auto = y_test

# Print the shape of compressed OES
print('X_train_auto:', X_train_auto.shape)
print('-----------------------------------------------')
print('X_test_auto:', X_train_auto.shape)
print('-----------------------------------------------')

X_train_auto: (1620, 10)
-----------------------------------------------
X_test_auto: (1620, 10)
-----------------------------------------------


## Random forest hyperparameters selection by 5-fold cross-validation and Bayesian optimization

In [5]:
# Necessary packages installation and importing
! pip install scikit-optimize
import skopt
from skopt import gbrt_minimize, gp_minimize
from skopt.utils import use_named_args
from skopt.space import Real, Categorical, Integer 

import sklearn.model_selection
from sklearn.ensemble import RandomForestClassifier

### Bayesian Optimization Structure

In [12]:
# Define the hyparameters of Random forest to be tuned
dim_feature = Integer(low=1, high=10, prior='log-uniform',name='feature')
dim_number = Integer(low=50,high=10000, prior='uniform',name='number')
dim_depth = Integer(low=1, high=60, prior='log-uniform',name='depth')

dimensions = [dim_feature,
              dim_number,
              dim_depth]
# initial value, from which the Bayesian optimization starts
default_parameters = [2,60,1]

In [14]:
# Define the objective to be minimized by Bayesian Optimization.
# Here, the objective is the accuracy of prediction. 
# For example, if the true lable is (1,1,1), i.e all the three transitions exist, while the prediction is (1,0,1),
# then this prediction will be treated as FALSE.
# The accuracy is the percentage of TRUE.

# Here 5-fold cross-validation is integrated into the objective function, 
# making the found hyperparameters more general and accurate.
# 5-fold cross-validation is extremely useful when the data is small.

@use_named_args(dimensions=dimensions)
def random_multi_acc(feature,number,depth):
    splits_n = 5

    kf = sklearn.model_selection.KFold(n_splits=splits_n,random_state=42, shuffle=True)
    kf.get_n_splits(X_train_auto)
    
    score = 0
    for train_index, test_index in kf.split(X_train_auto):

            RF = RandomForestClassifier(n_estimators=number,max_features=feature,max_depth=depth).fit(X_train_auto[train_index,:], 
                                                  Y_train_auto[train_index,:])
            score += RF.score(X_train_auto[test_index,:], Y_train_auto[test_index,:])
    print(score/5)
    return -score/5

In [ ]:
# use CheckpointSaver to save the optimization process in case of any interruption 
# This Bayesian optimization might take some time.
from skopt.callbacks import CheckpointSaver

checkpoint_saver = CheckpointSaver("./checkpoint.pkl", compress=9) # keyword arguments will be passed to `skopt.dump`

# last_res = skopt.load('checkpoint.pkl')

gp_result = gp_minimize(func=random_multi_acc,
                            dimensions=dimensions,
                            n_calls=400,
                            noise= 1e-8,
                            n_jobs=-1,
                            acq_func="EI",
                            x0=default_parameters,
                            #if starting from last checkpoint, comment out x0 = default_parameters,
                            #add x = last_res.x_iters, y = last_res.func_vals
                            callback=[checkpoint_saver],
                       )
opt_h = gp_result.x[0],gp_result.x[1],gp_result.x[2]
print(opt_h)
print(gp_result.fun)

In [39]:
# save the finished Bayesian optimization
# skopt.dump(gp_result,'BO_result')

In [25]:
# Here we directly load the completed Bayesian Optimization, with 400 evaluations.
# Note, you may obtain different hyperparameter values, but the score should be close.
BO_result = skopt.load('BO_result')

# Print the found best accuracy
print('Bestf found accuracy:', -BO_result.fun)
print('-----------------------------------------------')
# Print the corresponding hyperparameter values
print('Hyperparameter values:', BO_result.x)
print('-----------------------------------------------')
print('Corresponding to:','\n',
      'Features at each decision tree node:', BO_result.x[0],'\n',
      'Number of decision trees in the Random forest:', BO_result.x[1],'\n',
      'Maximal depth of each decision tree:', BO_result.x[2],'\n',)

Bestf found accuracy: 0.6876543209876542
-----------------------------------------------
Hyperparameter values: [7, 5854, 32]
-----------------------------------------------
Corresponding to: 
 Features at each decision tree node: 7 
 Number of decision trees in the Random forest: 5854 
 Maximal depth of each decision tree: 32 



### Examine the predition accuracy for each transition using the never-seen test set

In [6]:
# Use the confusion_matrix, which tells the numbers of true positive, false position, false negative and true negative.
from sklearn.metrics import confusion_matrix

# Train the Random forest using all the training data set with the found hyperparameters
RSs = RandomForestClassifier(n_estimators=5854,max_features=7,max_depth=32).fit(X_train_auto, Y_train_auto)

In [ ]:
RSs.score(X_train_auto[test_index,:], Y_train_auto[test_index,:])

In [7]:
# Calculate the confusion matrix for each transition using the never-seen test set
transition_name = ['N2(1+)', 'N2(2+)', 'NO Gamma']
for _ in range(3):
    c_tn, c_fp, c_fn, c_tp = confusion_matrix(RSs.predict(X_test_auto)[:,_],
                                              Y_test_auto[:,_]).ravel()
    print('For', transition_name[_], ':')
    print('TN:',c_tn,'FP:', c_fp,'FN:', c_fn,'TP:', c_tp)

For N2(1+) :
TN: 45 FP: 17 FN: 9 TP: 109
For N2(2+) :
TN: 54 FP: 18 FN: 4 TP: 104
For NO Gamma :
TN: 46 FP: 8 FN: 8 TP: 118


In [37]:
# Overall, the accuracy for each transition is high.
# It is noticeable that the accuracies of N2(1+) and N2(2+) are lower than that of NO Gamma.
# The reason is that in some OES simulations, the concentration of NO is unusually high.
# While the NO Gamma transition is extremely strong, making the transitions of N2(1+) and N2(2+) unrecognisable.